In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import skew
from scipy.stats import kurtosis
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
import math
import statsmodels.api as sm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read dataset
data = pd.read_excel("/Users/chaozhou/Desktop/ UCLA MFE/AFP/Report data.xlsx",'Report Data')

In [3]:
#calculation of conditional probability for different CEO periods
#first divide data into slices based on CEO years and time
data1_3 = data[(data['CEO Tenure in years']<2) & (data['Month'] == 3)]
data1_6 = data[(data['CEO Tenure in years']<2) & (data['Month'] == 6)]
data1_9= data[(data['CEO Tenure in years']<2) & (data['Month'] == 9)]
data1_12= data[(data['CEO Tenure in years']<2) & (data['Month'] == 12)]

data2_3 = data[(data['CEO Tenure in years']<5) & (data['CEO Tenure in years'] >= 2) & (data['Month'] == 3)]
data2_6 = data[(data['CEO Tenure in years']<5) & (data['CEO Tenure in years'] >= 2) & (data['Month'] == 6)]
data2_9= data[(data['CEO Tenure in years']<5) & (data['CEO Tenure in years'] >= 2) & (data['Month'] == 9)]
data2_12= data[(data['CEO Tenure in years']<5) & (data['CEO Tenure in years'] >= 2) & (data['Month'] == 12)]

data3_3 = data[(data['CEO Tenure in years']<8) & (data['CEO Tenure in years'] >= 5) & (data['Month'] == 3)]
data3_6 = data[(data['CEO Tenure in years']<8) & (data['CEO Tenure in years'] >= 5) & (data['Month'] == 6)]
data3_9= data[(data['CEO Tenure in years']<8) & (data['CEO Tenure in years'] >= 5) & (data['Month'] == 9)]
data3_12= data[(data['CEO Tenure in years']<8) & (data['CEO Tenure in years'] >= 5) & (data['Month'] == 12)]

data4_3 = data[(data['CEO Tenure in years']>=8) & (data['Month'] == 3)]
data4_6 = data[(data['CEO Tenure in years']>=8) & (data['Month'] == 6)]
data4_9= data[(data['CEO Tenure in years']>=8) & (data['Month'] == 9)]
data4_12= data[(data['CEO Tenure in years']>=8) & (data['Month'] == 12)]

data1_list = []
data2_list = []
data3_list = []
data4_list = []

data1_list.append(data1_3)
data1_list.append(data1_6)
data1_list.append(data1_9)
data1_list.append(data1_12)

data2_list.append(data2_3)
data2_list.append(data2_6)
data2_list.append(data2_9)
data2_list.append(data2_12)

data3_list.append(data3_3)
data3_list.append(data3_6)
data3_list.append(data3_9)
data3_list.append(data3_12)

data4_list.append(data4_3)
data4_list.append(data4_6)
data4_list.append(data4_9)
data4_list.append(data4_12)

states = ['EBITDA Margin YoY', 'FCF','Sales YoY', 'MA and Divestitures' , 'Restructuring as % of sales',
          'CapEx growth','Change in CEO', 'Company acquired']

In [4]:
#Calculation of conditional probability for CEO years <2
pro_list = []
name_list = []
for data in data1_list:
    for k in states:
        if k == 'EBITDA Margin YoY':
            data_original = data[data[k] < 0.01]
        elif k == 'FCF':
            data_original = data[data[k] < 0]
        elif k == 'Sales YoY':
            data_original = data[data[k] < 0]
        elif k == 'MA and Divestitures':
            data_original = data[data[k] > 0.02]
        elif k == 'Restructuring as % of sales':
            data_original = data[data[k] > 0.02]
        elif k == 'CapEx growth':
            data_original = data[data[k] < -0.1]
        elif k == 'Change in CEO':
            data_original = data[data[k] != 0]
        elif k == 'Company acquired':
            data_original = data[data[k] != 0]
        for j in states:
            if len(data_original) == 0:
                if (k == 'Company acquired') & (j == 'Company acquired'):
                    pro_H = 1
                    pro_list.append(pro_H)
                    name_list.append(k+" to "+ j)
                else:
                    pro_list.append(0)
                    name_list.append(k+" to "+ j)
            else:
                time = pd.to_datetime(data_original['Date']).reset_index(drop=True)
                time_1 = [time.iloc[i]+relativedelta(years = 1) for i in range(0,len(time))]
                df = pd.DataFrame({'Ticker':data_original['Ticker'],'Date':time_1}).reset_index(drop = True)
                temp = []
                for i in range(0,len(df)):
                    xzy = data_original.loc[(data['Ticker'] == df.iloc[i]['Ticker']) & (data['Date'] == df.iloc[i]['Date'])]
                    temp.append(xzy)
                if len(temp) != 0:
                    data_new = pd.concat(temp)
                else:
                    date_new = pd.DataFrame()   
                if j == 'EBITDA Margin YoY':
                    pro_A = len(data_new[data_new[j] < 0.01]) / len(data_original)
                    pro_list.append(pro_A)
                    name_list.append(k+" to "+ j)
                elif j == 'FCF':
                    pro_B = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_B)
                    name_list.append(k+" to "+ j)
                elif j == 'Sales YoY':
                    pro_C = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_C)
                    name_list.append(k+" to "+ j)
                elif j == 'MA and Divestitures':
                    pro_D = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_D)
                    name_list.append(k+" to "+ j)
                elif j == 'Restructuring as % of sales':
                    pro_E = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_E)
                    name_list.append(k+" to "+ j)
                elif j == 'CapEx growth':
                    pro_F = len(data_new[data_new[j] < -0.1]) / len(data_original)
                    pro_list.append(pro_F)
                    name_list.append(k+" to "+ j)
                elif j == 'Change in CEO':
                    pro_G = len(data_new[data_new[j] != 0]) / len(data_original)
                    pro_list.append(pro_G)
                    name_list.append(k+" to "+ j)
                elif j == 'Company acquired':
                    if k == 'Company acquired':
                        pro_H = 1
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)
                    else:
                        pro_H = len(data_new[data_new[j] != 0]) / len(data_original)
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)                 
table = pd.DataFrame({'Category':name_list,'Probability':pro_list}).reset_index(drop = True)
table_ave_1 = table.groupby(['Category'],as_index=False).mean().sort_values(by=['Category'])

In [5]:
#Calculation of conditional probability for CEO years <2
pro_list = []
name_list = []
for data in data2_list:
    for k in states:
        if k == 'EBITDA Margin YoY':
            data_original = data[data[k] < 0.01]
        elif k == 'FCF':
            data_original = data[data[k] < 0]
        elif k == 'Sales YoY':
            data_original = data[data[k] < 0]
        elif k == 'MA and Divestitures':
            data_original = data[data[k] > 0.02]
        elif k == 'Restructuring as % of sales':
            data_original = data[data[k] > 0.02]
        elif k == 'CapEx growth':
            data_original = data[data[k] < -0.1]
        elif k == 'Change in CEO':
            data_original = data[data[k] != 0]
        elif k == 'Company acquired':
            data_original = data[data[k] != 0]
        for j in states:
            if len(data_original) == 0:
                if (k == 'Company acquired') & (j == 'Company acquired'):
                    pro_H = 1
                    pro_list.append(pro_H)
                    name_list.append(k+" to "+ j)
                else:
                    pro_list.append(0)
                    name_list.append(k+" to "+ j)
            else:
                time = pd.to_datetime(data_original['Date']).reset_index(drop=True)
                time_1 = [time.iloc[i]+relativedelta(years = 1) for i in range(0,len(time))]
                df = pd.DataFrame({'Ticker':data_original['Ticker'],'Date':time_1}).reset_index(drop = True)
                temp = []
                for i in range(0,len(df)):
                    xzy = data_original.loc[(data['Ticker'] == df.iloc[i]['Ticker']) & (data['Date'] == df.iloc[i]['Date'])]
                    temp.append(xzy)
                if len(temp) != 0:
                    data_new = pd.concat(temp)
                else:
                    date_new = pd.DataFrame()   
                if j == 'EBITDA Margin YoY':
                    pro_A = len(data_new[data_new[j] < 0.01]) / len(data_original)
                    pro_list.append(pro_A)
                    name_list.append(k+" to "+ j)
                elif j == 'FCF':
                    pro_B = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_B)
                    name_list.append(k+" to "+ j)
                elif j == 'Sales YoY':
                    pro_C = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_C)
                    name_list.append(k+" to "+ j)
                elif j == 'MA and Divestitures':
                    pro_D = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_D)
                    name_list.append(k+" to "+ j)
                elif j == 'Restructuring as % of sales':
                    pro_E = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_E)
                    name_list.append(k+" to "+ j)
                elif j == 'CapEx growth':
                    pro_F = len(data_new[data_new[j] < -0.1]) / len(data_original)
                    pro_list.append(pro_F)
                    name_list.append(k+" to "+ j)
                elif j == 'Change in CEO':
                    pro_G = len(data_new[data_new[j] != 0]) / len(data_original)
                    pro_list.append(pro_G)
                    name_list.append(k+" to "+ j)
                elif j == 'Company acquired':
                    if k == 'Company acquired':
                        pro_H = 1
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)
                    else:
                        pro_H = len(data_new[data_new[j] != 0]) / len(data_original)
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)                 
table = pd.DataFrame({'Category':name_list,'Probability':pro_list}).reset_index(drop = True)
table_ave_2 = table.groupby(['Category'],as_index=False).mean().sort_values(by=['Category'])

In [6]:
#Calculation of conditional probability for CEO years <2
pro_list = []
name_list = []
for data in data3_list:
    for k in states:
        if k == 'EBITDA Margin YoY':
            data_original = data[data[k] < 0.01]
        elif k == 'FCF':
            data_original = data[data[k] < 0]
        elif k == 'Sales YoY':
            data_original = data[data[k] < 0]
        elif k == 'MA and Divestitures':
            data_original = data[data[k] > 0.02]
        elif k == 'Restructuring as % of sales':
            data_original = data[data[k] > 0.02]
        elif k == 'CapEx growth':
            data_original = data[data[k] < -0.1]
        elif k == 'Change in CEO':
            data_original = data[data[k] != 0]
        elif k == 'Company acquired':
            data_original = data[data[k] != 0]
        for j in states:
            if len(data_original) == 0:
                if (k == 'Company acquired') & (j == 'Company acquired'):
                    pro_H = 1
                    pro_list.append(pro_H)
                    name_list.append(k+" to "+ j)
                else:
                    pro_list.append(0)
                    name_list.append(k+" to "+ j)
            else:
                time = pd.to_datetime(data_original['Date']).reset_index(drop=True)
                time_1 = [time.iloc[i]+relativedelta(years = 1) for i in range(0,len(time))]
                df = pd.DataFrame({'Ticker':data_original['Ticker'],'Date':time_1}).reset_index(drop = True)
                temp = []
                for i in range(0,len(df)):
                    xzy = data_original.loc[(data['Ticker'] == df.iloc[i]['Ticker']) & (data['Date'] == df.iloc[i]['Date'])]
                    temp.append(xzy)
                if len(temp) != 0:
                    data_new = pd.concat(temp)
                else:
                    date_new = pd.DataFrame()   
                if j == 'EBITDA Margin YoY':
                    pro_A = len(data_new[data_new[j] < 0.01]) / len(data_original)
                    pro_list.append(pro_A)
                    name_list.append(k+" to "+ j)
                elif j == 'FCF':
                    pro_B = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_B)
                    name_list.append(k+" to "+ j)
                elif j == 'Sales YoY':
                    pro_C = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_C)
                    name_list.append(k+" to "+ j)
                elif j == 'MA and Divestitures':
                    pro_D = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_D)
                    name_list.append(k+" to "+ j)
                elif j == 'Restructuring as % of sales':
                    pro_E = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_E)
                    name_list.append(k+" to "+ j)
                elif j == 'CapEx growth':
                    pro_F = len(data_new[data_new[j] < -0.1]) / len(data_original)
                    pro_list.append(pro_F)
                    name_list.append(k+" to "+ j)
                elif j == 'Change in CEO':
                    pro_G = len(data_new[data_new[j] != 0]) / len(data_original)
                    pro_list.append(pro_G)
                    name_list.append(k+" to "+ j)
                elif j == 'Company acquired':
                    if k == 'Company acquired':
                        pro_H = 1
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)
                    else:
                        pro_H = len(data_new[data_new[j] != 0]) / len(data_original)
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)                 
table = pd.DataFrame({'Category':name_list,'Probability':pro_list}).reset_index(drop = True)
table_ave_3 = table.groupby(['Category'],as_index=False).mean().sort_values(by=['Category'])

In [7]:
#Calculation of conditional probability for CEO years <2
pro_list = []
name_list = []
for data in data4_list:
    for k in states:
        if k == 'EBITDA Margin YoY':
            data_original = data[data[k] < 0.01]
        elif k == 'FCF':
            data_original = data[data[k] < 0]
        elif k == 'Sales YoY':
            data_original = data[data[k] < 0]
        elif k == 'MA and Divestitures':
            data_original = data[data[k] > 0.02]
        elif k == 'Restructuring as % of sales':
            data_original = data[data[k] > 0.02]
        elif k == 'CapEx growth':
            data_original = data[data[k] < -0.1]
        elif k == 'Change in CEO':
            data_original = data[data[k] != 0]
        elif k == 'Company acquired':
            data_original = data[data[k] != 0]
        for j in states:
            if len(data_original) == 0:
                if (k == 'Company acquired') & (j == 'Company acquired'):
                    pro_H = 1
                    pro_list.append(pro_H)
                    name_list.append(k+" to "+ j)
                else:
                    pro_list.append(0)
                    name_list.append(k+" to "+ j)
            else:
                time = pd.to_datetime(data_original['Date']).reset_index(drop=True)
                time_1 = [time.iloc[i]+relativedelta(years = 1) for i in range(0,len(time))]
                df = pd.DataFrame({'Ticker':data_original['Ticker'],'Date':time_1}).reset_index(drop = True)
                temp = []
                for i in range(0,len(df)):
                    xzy = data_original.loc[(data['Ticker'] == df.iloc[i]['Ticker']) & (data['Date'] == df.iloc[i]['Date'])]
                    temp.append(xzy)
                if len(temp) != 0:
                    data_new = pd.concat(temp)
                else:
                    date_new = pd.DataFrame()   
                if j == 'EBITDA Margin YoY':
                    pro_A = len(data_new[data_new[j] < 0.01]) / len(data_original)
                    pro_list.append(pro_A)
                    name_list.append(k+" to "+ j)
                elif j == 'FCF':
                    pro_B = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_B)
                    name_list.append(k+" to "+ j)
                elif j == 'Sales YoY':
                    pro_C = len(data_new[data_new[j] < 0]) / len(data_original)
                    pro_list.append(pro_C)
                    name_list.append(k+" to "+ j)
                elif j == 'MA and Divestitures':
                    pro_D = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_D)
                    name_list.append(k+" to "+ j)
                elif j == 'Restructuring as % of sales':
                    pro_E = len(data_new[data_new[j] > 0.02]) / len(data_original)
                    pro_list.append(pro_E)
                    name_list.append(k+" to "+ j)
                elif j == 'CapEx growth':
                    pro_F = len(data_new[data_new[j] < -0.1]) / len(data_original)
                    pro_list.append(pro_F)
                    name_list.append(k+" to "+ j)
                elif j == 'Change in CEO':
                    pro_G = len(data_new[data_new[j] != 0]) / len(data_original)
                    pro_list.append(pro_G)
                    name_list.append(k+" to "+ j)
                elif j == 'Company acquired':
                    if k == 'Company acquired':
                        pro_H = 1
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)
                    else:
                        pro_H = len(data_new[data_new[j] != 0]) / len(data_original)
                        pro_list.append(pro_H)
                        name_list.append(k+" to "+ j)                 
table = pd.DataFrame({'Category':name_list,'Probability':pro_list}).reset_index(drop = True)
table_ave_4 = table.groupby(['Category'],as_index=False).mean().sort_values(by=['Category'])

In [8]:
transitionMatrix1 = []
cardinality = 8
for i in range(0,cardinality):
    row = list(table_ave_1['Probability'].iloc[i*cardinality:(i+1)*cardinality])
    transitionMatrix1.append(row)

In [9]:
transitionMatrix2 = []
cardinality = 8
for i in range(0,cardinality):
    row = list(table_ave_2['Probability'].iloc[i*cardinality:(i+1)*cardinality])
    transitionMatrix2.append(row)

In [10]:
transitionMatrix3 = []
cardinality = 8
for i in range(0,cardinality):
    row = list(table_ave_3['Probability'].iloc[i*cardinality:(i+1)*cardinality])
    transitionMatrix3.append(row)

In [11]:
transitionMatrix4 = []
cardinality = 8
for i in range(0,cardinality):
    row = list(table_ave_4['Probability'].iloc[i*cardinality:(i+1)*cardinality])
    transitionMatrix4.append(row)